# Exploración de textos con RAGS

Documento completo en https://tecnologiaysociedad.substack.com/

### pre procesamiento

In [ ]:
import pandas as pd
import re

df = pd.read_csv("letras.csv") #cargamos el csv.

def limpiar_texto(texto):
    texto = str(texto).lower().strip()
    texto = re.sub(r'\s+', ' ', texto)  # Reemplazo de espacios múltiples
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminación de signos de puntuación
    return texto

df["letra_limpia"] = df["Letra"].apply(limpiar_texto) #limpiamos

print("Preprocesamiento completado.")


Preprocesamiento completado.


### Generacion de embeddings

In [4]:
from sentence_transformers import SentenceTransformer

modelo_embeddings = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

df["embedding"] = df["letra_limpia"].apply(lambda x: modelo_embeddings.encode(x).tolist())

print("Embeddings generados.")


c:\Users\Pedro\Desktop\proyectos_25\introllm\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embeddings generados.


In [5]:
for idx, row in df.iterrows():
    print(row)  # Verifica qué datos hay en cada fila
    print(row["embedding"])  # Asegura que la columna está accesible


Álbum                                  Yendo De La Cama Al Living
Título                                 Yendo De La Cama Al Living
Letra           Podés pasear en limousine, cortar las flores d...
letra_limpia    podés pasear en limousine cortar las flores de...
embedding       [-0.10412989556789398, 0.04328650236129761, -0...
Name: 0, dtype: object
[-0.10412989556789398, 0.04328650236129761, -0.33559346199035645, 0.22983868420124054, 0.32912373542785645, -0.14184534549713135, -0.031385164707899094, 0.3195156157016754, -0.09695079922676086, 0.14614982903003693, -0.13177621364593506, -0.18813946843147278, 0.26114949584007263, -0.20945920050144196, 0.07133938372135162, -0.23845219612121582, 0.04638542979955673, 0.060892440378665924, -0.016897648572921753, -0.2566034197807312, -0.34727075695991516, -0.008667081594467163, -0.0991017296910286, 0.2880050241947174, -0.29059648513793945, -0.044842734932899475, 0.09689432382583618, -0.0798950269818306, -0.07254716008901596, 0.070338174700737, 

### Almacenamiento en ChromaDB

In [ ]:
import chromadb

# Inicializar ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="letras_vectorizadas")

# Insertar los embeddings junto con los metadatos
for idx, row in df.iterrows():
    collection.add(
        ids=[str(idx)],
        embeddings=[row["embedding"]],
        metadatas=[{"album": row["Álbum"], "titulo": row["Título"], "letra": row["letra_limpia"]}] #definimos los metadatos
    )

print("Embeddings almacenados en ChromaDB.")


### Funcion de busqueda semantica y recuperacion

In [7]:
def buscar_letras(query, n=3):
    """
    Realiza una búsqueda semántica en la base vectorizada de letras de canciones.
    
    Parámetros:
    query (str): Consulta en lenguaje natural, como un concepto o una emoción.
    n (int): Número de resultados a devolver (por defecto 3).
    """

    # Convertimos la consulta en un embedding utilizando el mismo modelo que generó los embeddings de las letras
    query_embedding = modelo_embeddings.encode(query).tolist()

    # Realizamos la consulta en ChromaDB, buscando los 'n' fragmentos más similares a la consulta
    resultados = collection.query(
        query_embeddings=[query_embedding],  # Embedding de la consulta
        n_results=n  # Número de resultados a recuperar
    )

    # Iteramos sobre los resultados obtenidos y mostramos la información
    for i, r in enumerate(resultados["documents"][0]):  
        print(f"\n{i+1}. {resultados['metadatas'][0][i]['titulo']} ({resultados['metadatas'][0][i]['album']})")  
        # Se muestra el título de la canción y el álbum al que pertenece
        
        print(f"{resultados['metadatas'][0][i]['letra'][:500]}...\n")  
        # Se imprime un fragmento de los primeros 500 caracteres de la letra recuperada

# Ejemplo de uso: buscar canciones que hablen sobre "amor y algarabía"
buscar_letras("amor y algarabía", n=3)



1. Tuve Tu Amor (Piano Bar)
tuve tu amor tuve tu amor en mí ya tuve tu amor tú ves tu amor tú ves tu amor en mí ya tuve tu amor el exilio te vistió de actriz porque alguien se confundió alguien te descubrió cansado de esperar cómo volverte a ver alguien se suicidó alguien te traicionó los sitios girarán por siempre sitios para transgredir hay algo en tu nariz que escondes muy bien te encuentro feliztriste susana se escapó miguel también ana se quedó ana se quedó te veo bien te veo muy bien te ves muy bien te ves muy bien t...


2. Necesito Tu Amor (Parte de la Religión)
voy a cambiar voy a insistir voy a pelear voy a seguir yo necesito tu amor tu amor me salva y me sirve yo necesito tu amor cada día un poco más yo tengo el vicio de dejarme llevar y poner mi cabeza en marte tengo prejuicios que no puedo sacar tengo un cuerpo que quiere amarte yo ví tu amor oooh yo ví tu amor oooh dentro del mal cerca del fin cerca de vos dentro de mí yo necesito tu amor tu amor me salva y me sirve yo n

## Rag en acción

### Instanciamos gemini

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

# Instanciar modelo Gemini (desarrollo)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    google_api_key="apikey"
)

### Creamos en propt de analisis

In [ ]:
import random
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

# Instancia del modelo
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    google_api_key="apikey"
)

# Dimensiones analíticas refinadas
dimensiones = {
    "economía afectiva": "¿Cómo se distribuyen las emociones en el texto? ¿Qué figuras del deseo, la pérdida o la obsesión se articulan? ¿Qué intensidad asumen?",
    "posicionamiento discursivo": "¿Quién habla y desde dónde? ¿Se asume un rol dominante, vulnerable, espectral? ¿Cómo se posiciona la voz en relación al otro, al amor, al mundo?",
    "sistema de imágenes": "¿Qué léxicos simbólicos componen la representación del amor? ¿Qué metáforas lo recorren? ¿Qué registros (tecnológico, religioso, urbano, corporal) estructuran su forma textual?"
}

# Prompt estructurado, pero sin forzar JSON
template = """
Estás analizando un conjunto de 15 letras completas de canciones de Charly García (1982–1990), seleccionadas por su cercanía semántica al tema: "{tema}".

Tu tarea es examinar estas letras desde la siguiente dimensión: "{dimension}". 
Descripción: {descripcion}

Estructurá tu análisis en tres partes:
1. Una síntesis conceptual general.
2. Ejemplos textuales relevantes (citas).
3. Observaciones sobre estilo, tono o estructura.

No hagas títulos ni markdown. Solo separá cada parte con una línea en blanco.

Letra(s):
{letras}

Análisis:
"""

prompt = PromptTemplate.from_template(template)

analisis_chain = (
    {
        "tema": RunnableLambda(lambda x: x["tema"]),
        "dimension": RunnableLambda(lambda x: x["dimension"]),
        "descripcion": RunnableLambda(lambda x: x["descripcion"]),
        "letras": RunnableLambda(lambda x: x["letras"])
    }
    | prompt
    | llm
)

# Función principal
def analisis_tematico_estructurado(tema: str, n: int = 15):
    embedding = modelo_embeddings.encode(tema).tolist()

    resultados = collection.query(
        query_embeddings=[embedding],
        n_results=n
    )

    letras = "\n---\n".join([
        meta["letra"] for meta in resultados["metadatas"][0]
        if meta.get("letra")
    ])

    salida = {
        "tema": tema,
        "dimensiones": {}
    }

    for dim, desc in dimensiones.items():
        result = analisis_chain.invoke({
            "tema": tema,
            "dimension": dim,
            "descripcion": desc,
            "letras": letras
        })
        salida["dimensiones"][dim] = result.content.strip()

    return salida

# Ejecutar análisis
resultado_json = analisis_tematico_estructurado("amor")

# Convertir a DataFrame plano
df_raw = pd.DataFrame.from_dict(resultado_json["dimensiones"], orient="index", columns=["respuesta"])
df_raw.reset_index(inplace=True)
df_raw.rename(columns={"index": "dimension"}, inplace=True)

# Parsear las 3 partes separadas por líneas en blanco
df_split = df_raw.copy()
df_split[['síntesis', 'ejemplos', 'observaciones']] = df_raw['respuesta'].str.split(r'\n\s*\n', n=2, expand=True)
df_split.drop(columns=["respuesta"], inplace=True)

# Mostrar
print(df_split)


                    dimension  \
0           economía afectiva   
1  posicionamiento discursivo   
2         sistema de imágenes   

                                            síntesis  \
0  En el universo lírico de Charly García, el amo...   
1  El posicionamiento discursivo en las letras de...   
2  El sistema de imágenes del amor en las letras ...   

                                            ejemplos  \
0  "Yo necesito tu amor tu amor me salva y me sir...   
1  "yo necesito tu amor tu amor me salva y me sir...   
2                                Ejemplos textuales:   

                                       observaciones  
0  El estilo de García se caracteriza por la yuxt...  
1  El estilo de las letras es ecléctico y experim...  
2  *   "yo necesito tu amor tu amor me salva y me...  
